In [163]:
from neo4j import GraphDatabase
import pandas as pd

In [164]:
# connexion à la BDD
uri             = "bolt://localhost:7687"
userName        = "neo4j"
password        = "neo4j"


graphDB_Driver = GraphDatabase.driver(uri, auth=(userName, password))

In [165]:
# chargement du csv
csv_file_path = "Data/communes-departement-region.csv"
chunk_size = 1000
chunks = []

for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size):
    chunks.append(chunk)
    # break
    
df = pd.concat(chunks)
df.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [166]:
# création d'un dataframe région
regions = df[['code_region', 'nom_region']].drop_duplicates()
regions

,code_region,nom_region
0,84.0,Auvergne-Rhône-Alpes
457,32.0,Hauts-de-France
1608,93.0,Provence-Alpes-Côte d'Azur
2582,44.0,Grand Est
3086,76.0,Occitanie
4861,28.0,Normandie
5889,75.0,Nouvelle-Aquitaine
6823,24.0,Centre-Val de Loire
7406,27.0,Bourgogne-Franche-Comté
8124,53.0,Bretagne


In [167]:
# création d'un dataframe département
departements = df[['code_departement', 'nom_departement', 'code_region']].drop_duplicates()
departements

,code_departement,nom_departement,code_region
0,1,Ain,84.0
457,2,Aisne,32.0
1287,3,Allier,84.0
1608,4,Alpes-de-Haute-Provence,93.0
1852,5,Hautes-Alpes,93.0
...,...,...,...
38911,97,NaN,NaN
38914,976,Mayotte,6.0
38939,98,NaN,NaN
39162,NaN,NaN,NaN


In [168]:
# création d'un dataframe ville
villes = df[['code_commune_INSEE', 'nom_commune_postal', 'code_postal', 'libelle_acheminement', 'latitude', 'longitude', 'code_commune', 'nom_commune_complet', 'code_departement']].drop_duplicates()
villes

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,latitude,longitude,code_commune,nom_commune_complet,code_departement
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,46.153426,4.926114,1.0,L'Abergement-Clémenciat,1
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,46.009188,5.428017,2.0,L'Abergement-de-Varey,1
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,45.960848,5.372926,4.0,Ambérieu-en-Bugey,1
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,45.996180,4.912273,5.0,Ambérieux-en-Dombes,1
4,1006,AMBLEON,1300,AMBLEON,45.749499,5.594320,6.0,Ambléon,1
...,...,...,...,...,...,...,...,...,...
39196,98831,VOH,98883,OUACO,NaN,NaN,831.0,Voh,98.0
39197,98832,YATE,98834,YATE,NaN,NaN,832.0,Yate,98.0
39198,98833,KOUAOUA,98818,KOUAOUA,NaN,NaN,833.0,Kouaoua,98.0
39199,98901,ILE DE CLIPPERTON,98799,ILE DE CLIPPERTON,NaN,NaN,901.0,Ile de clipperton,98.0


In [169]:
def reset_tables():
    """
    Vide la base de données de toutes les régions, départements et villes
    """
    with graphDB_Driver.session() as session:
        session.run("MATCH (n:Region) DETACH DELETE n")
        session.run("MATCH (n:Department) DETACH DELETE n")
        session.run("MATCH (n:Ville) DETACH DELETE n")

# Call the function to reset tables
reset_tables()

In [170]:
def create_region_node(dfRegions):
    """
    Créé les nœuds région à partir d'un dataframe
    """
    for region in dfRegions.index:
        with graphDB_Driver.session() as session:
            try:
                code_region = int(dfRegions.loc[region, "code_region"])
                region_name = dfRegions.loc[region, "nom_region"]
                query = (
                    "MERGE (r:Region {code: $codeRegion, name: $regionName})"
                )
                session.run(query, codeRegion=code_region, regionName=region_name)
            except:
                pass

create_region_node(regions)


In [171]:
def create_departments(dfDepartement):
    """
    Créé les nœuds département à partir d'un dataframe et les relie aux régions
    """
    with graphDB_Driver.session() as session:
        for department in dfDepartement.index:
            try:
                code_departement = int(dfDepartement.loc[department, "code_departement"])
                code_region = int(dfDepartement.loc[department, "code_region"])
                departement_name = dfDepartement.loc[department, "nom_departement"]
                query = (
                    "MERGE (d:Department {code: $code_departement, name: $departement_name})"
                )
                session.run(query, code_departement=code_departement, departement_name=departement_name)
                relationship_query = (
                    "MATCH (d:Department {code: $departmentCode})"
                    " MATCH (r:Region {code: $regionCode})"
                    " MERGE (d)-[:BELONGS_TO]->(r)"
                )
                session.run(relationship_query, departmentCode=code_departement, regionCode=code_region)
            except:
                pass


create_departments(departements)

In [172]:
def create_cities(dfCities):
    """
    Créé les nœuds ville à partir d'un dataframe et les relie aux départements
    """
    with graphDB_Driver.session() as session:
        for _, city_data in dfCities.iterrows():
            query = (
                "MERGE (v:Ville {name: $nom_commune_postal, "
                "code_commune_INSEE: $code_commune_INSEE, "
                "code_postal: $code_postal, "
                "libelle_acheminement: $libelle_acheminement, "
                "latitude: $latitude, "
                "longitude: $longitude, "
                "code_commune: $code_commune, "
                "nom_commune_complet: $nom_commune_complet})"
            )
            session.run(query, **city_data.to_dict())
            
            relationship_query = (
                "MATCH (v:Ville {code_commune_INSEE: $code_commune_INSEE})"
                "MATCH (d:Department {code: $departmentCode})"
                "MERGE (v)-[:IN]->(d)"
            )
            session.run(relationship_query, code_commune_INSEE=city_data["code_commune_INSEE"], departmentCode=city_data["code_departement"])

create_cities(villes)

ClientError: {code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of NaN property value for 'latitude': (:Ville {latitude: NaN})}